In [ ]:
import numpy as np
from scipy import optimize, stats
from problem import Problem
from iminuit import Minuit

import matplotlib as mpl
import matplotlib.pyplot as plt

# Model Class

Here we set up a helper class which loads the data, model components, and model specification and allows us to calculate the $\chi^2$ for a given set of parameters.

In [ ]:
perseus = Problem('./Data/ReducedData/Perseus_Data.h5', # path to the data archive
                  './Data/ModelComponents/APEC_Grid.npz', # path to the saved APEC interpolation grid
                  './Data/ModelComponents/Absorption_Grid.npy', # path to the saved hydrogen absorption grid
                  './Fitting/E3/Perseus/Initial.npz',  # path to the model specification 
                  min_energy =3, # minimum energy in the emission frame which defines the energy ROI
                  max_energy = 6, # maximum energy in the emission frame which defines the energy ROI
                 )

In [ ]:
# Generating a random parameter vector
param_vector = stats.uniform.rvs(loc = perseus.bounds[0],
                                 scale = perseus.bounds[1] - perseus.bounds[0])

model = perseus.get_model(param_vector)
chi2 = perseus.fitness(param_vector)

print('ChiSq: ', chi2)
print('Degrees of Freedom:', len(perseus.cts[perseus.locs]) - len(param_vector))

In [ ]:
# Access the data for plotting
energies = (1+perseus.z)*perseus.Eout_data # Emission-frame energies
cts = perseus.cts # Number of counts in each bin
err = perseus.err # Gaussian error bars for each bin

model = perseus.get_model(param_vector) # The model associated with the specified parameter vector


# Plot the data and the model. By using `locs` we restrict to just plotting within our energy ROI.
plt.plot(energies[perseus.locs], model[perseus.locs], 
         color = 'black', label = 'Model')

plt.errorbar(energies[perseus.locs], cts[perseus.locs], yerr = err[perseus.locs],
             color = 'grey', fmt ='s', ms= 1, label = 'Data')

plt.xlabel('$E$ [keV]')
plt.ylabel('Counts')

plt.legend()
plt.tight_layout()

# Setting up an Optimization

The random parameter vector we generated was almost certainly a bad fit. Now we set up an optimization for minimizing the $\chi^2$.

Warning: The execution time for this code is very long!

In [ ]:
# Running a global optimization
out = optimize.differential_evolution(perseus.fitness, perseus.bounds.T, polish = False, 
                                      tol = 1e-4, atol = 0, popsize = 50, init = 'sobol',
                                      maxiter=100000)

# Polish with minuit
m = Minuit.from_array_func(perseus.fitness, param_vector, limit =perseus.bounds.T, errordef = 1)

Congratulations! You have fit the model. You should save here in the desired manner.